In [ ]:
"""
    SLNN/DNN code for ISR 
    manual code which you can customize the network details, 
    such as network architecture, learning rate, etc.
"""


In [ ]:
import time
import os.path
import numpy as np
import pandas as pd
import datetime as dt
import multiprocess as mp
# ML related 
import torch
from torch import nn
from torch.utils.data import DataLoader
import tensorflow as tf

# plot related
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import matplotlib.gridspec as gridspec

from glob import glob
from tqdm import tqdm
from collections import OrderedDict
# from multiprocessing import Pool


In [ ]:
#%% paths of source files [please adjust accordingly]

path_work_fd = './data'
path_figsav = './nel_daily'
path_mdsav = './md_res'


In [ ]:
#%% parameters 
parms_inv = ['year', 'dayno', 'ut', 'f10.7', 'ap3', 'nel']
parms_input = ['year', 'doy_sin', 'doy_cos', 'ut_sin', 'ut_cos', 'f10.7', 'ap3']
parms_inp_norm = ['year', 'f10.7', 'ap3']
parms_output = ['nel']

# split on training/val/test sets
list_yr_val = [2010, 2015]
list_yr_test = [2007, 2012]

# location of Millstone ISR
isr_lat, isr_lon = 42.61, 288.51
diff_utslt = abs(isr_lon-360)/360*24


In [ ]:
#%% confine to specified GPU device [please adjust accordingly]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


In [ ]:
#%% prepare data [please adjust accordingly] 
'''
    In our study, we had processed data in UT in order to compare with SLT, 
    you can customize for your convenience
    
'''
df_isr = pd.read_feather(os.path.join(path_work_fd, 'isr_hourly_ver1.lz4'))

df_isr.index = pd.to_datetime(df_isr['timestamp_ut'])
# prepare all the needed input parameters
df_isr['year'] = df_isr.index.year
df_isr['dayno'] = df_isr.index.dayofyear
df_isr['ut'] = df_isr.index.hour+df_isr.index.minute/60

# get cyclic on dayno and ut
doy_sin = (np.sin(df_isr['dayno']/365 * 2*np.pi)+1)/2
doy_cos = (np.cos(df_isr['dayno']/365 * 2*np.pi)+1)/2
ut_sin = (np.sin(df_isr['ut']/24 * 2*np.pi)+1)/2
ut_cos = (np.cos(df_isr['ut']/24 * 2*np.pi)+1)/2

df_isr_norm = df_isr.copy(True)
df_isr_norm['doy_sin'] = doy_sin
df_isr_norm['doy_cos'] = doy_cos
df_isr_norm['ut_sin'] = ut_sin
df_isr_norm['ut_cos'] = ut_cos
# normalize year, F10.7 and Ap3
df_isr_norm.loc[:, parms_inp_norm] /= df_isr.loc[:, parms_inp_norm].max()

# split training/validation/test
df_isr_train_norm = df_isr_norm.loc[~df_isr_norm.index.year.isin(list_yr_val+list_yr_test)]
df_isr_val_norm = df_isr_norm.loc[df_isr_norm.index.year.isin(list_yr_val)]
df_isr_test_norm = df_isr_norm.loc[df_isr_norm.index.year.isin(list_yr_test)]


In [ ]:
# set up the network [please adjust accordingly]
input_size = len(parms_input)
hidden_sizes = [18,36,38,54]
output_size = len(parms_output)
inp_hid = [input_size]+hidden_sizes

md_name = 'dnn_ak_run6'
bs_size = 64
lr_ = 5e-05
num_epoch = 8000

# make dir
os.makedirs(os.path.join(path_mdsav, md_name), exist_ok=True)

# make DataLoader according to batch size
train_X = torch.tensor(df_isr_train_norm.loc[:, parms_input].values, dtype=torch.float).to(device)
train_y = torch.tensor(df_isr_train_norm.loc[:, parms_output].values, dtype=torch.float).to(device)

val_X = torch.tensor(df_isr_val_norm.loc[:, parms_input].values, dtype=torch.float).to(device)
val_y = torch.tensor(df_isr_val_norm.loc[:, parms_output].values, dtype=torch.float).to(device)

test_X = torch.tensor(df_isr_test_norm.loc[:, parms_input].values, dtype=torch.float).to(device)
test_y = torch.tensor(df_isr_test_norm.loc[:, parms_output].values, dtype=torch.float).to(device)

# load into DataLoader
train_loader = DataLoader(list(zip(train_X, train_y)), shuffle=True, batch_size=bs_size)
val_loader = DataLoader(list(zip(val_X, val_y)), shuffle=True, batch_size=bs_size)
test_loader = DataLoader(list(zip(test_X, test_y)), shuffle=True, batch_size=bs_size)


# make the structure
list_dense = [(f'dense_{idx}', nn.Linear(inp_hid[idx], inp_hid[idx+1])) for idx, neu_num in enumerate(inp_hid[:-1])]
list_actv = [(f'activation_{idx}', nn.ReLU()) for idx in range(len(inp_hid)-1)]
dict_nn_structure = list(sum(list(zip(list_dense, list_actv)), ()))+[('output', nn.Linear(hidden_sizes[-1], 1))]


In [ ]:
# define the NN
model = nn.Sequential(OrderedDict(dict_nn_structure))
model.to(device)


In [ ]:
# continue the training
torch.manual_seed(49)

def weights_init(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight.data)
        torch.nn.init.constant_(m.bias.data, 0)
model.apply(weights_init)

loss_l1 = nn.L1Loss()
loss_mse = nn.MSELoss()

# Optimizers require the parameters to optimize and a learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr_)

list_loss_train, list_loss_val, list_loss_test = [], [], []

# iterate over the epochs ## save loss.item() only
stime = time.time()

for e in range(num_epoch):
    r_loss_train, r_rmse_train, r_re_train = 0, 0, 0
    # on training set
    model.train()
    for X_batch, y_batch in train_loader:
        y_pred = model(X_batch)
        loss_train = loss_l1(y_pred, y_batch)
        r_loss_train += float(loss_train.item())
        r_rmse_train += float(torch.sqrt(loss_mse(y_pred, y_batch)).item())
        r_re_train += float(torch.mean(torch.abs(y_pred-y_batch)/y_batch).item())
        # update the network
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
    # averaged training loss over all batches
    loss_avg_train = r_loss_train / len(train_loader)
    rmse_avg_train = r_rmse_train / len(train_loader)
    re_avg_train = r_re_train / len(train_loader)
        
    # on val set and test set on the whole set
    r_loss_val, r_rmse_val, r_re_val = 0, 0, 0
    r_loss_test, r_rmse_test, r_re_test = 0, 0, 0
    
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            y_pred = model(X_batch)
            loss_val = loss_l1(y_pred, y_batch).item()
            r_loss_val += float(loss_val)
            r_rmse_val += float(torch.sqrt(loss_mse(y_pred, y_batch)).item())
            r_re_val += float(torch.mean(torch.abs(y_pred-y_batch)/y_batch).item())            
            
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch)
            loss_test = loss_l1(y_pred, y_batch).item()
            r_loss_test += float(loss_test)
            r_rmse_test += float(torch.sqrt(loss_mse(y_pred, y_batch)).item())
            r_re_test += float(torch.mean(torch.abs(y_pred-y_batch)/y_batch).item())            
        
    loss_avg_val = r_loss_val / len(val_loader)
    rmse_avg_val = r_rmse_val / len(val_loader)
    re_avg_val = r_re_val / len(val_loader)
    
    loss_avg_test = r_loss_test / len(test_loader)
    rmse_avg_test = r_rmse_test / len(test_loader)
    re_avg_test = r_re_test / len(test_loader)

    errs = {}
    errs['mae'], errs['rmse'], errs['re'] = {}, {}, {}
    errs['mae']['train'] = loss_avg_train
    errs['mae']['val'] = loss_avg_val
    errs['mae']['test'] = loss_avg_test

    errs['rmse']['train'] = rmse_avg_train
    errs['rmse']['val'] = rmse_avg_val
    errs['rmse']['test'] = rmse_avg_test

    errs['re']['train'] = re_avg_train
    errs['re']['val'] = re_avg_val
    errs['re']['test'] = re_avg_test
    # display the info every 100 epochs
    if e%100 == 0:
        print(f'# Epoch: {e}, Training loss: {loss_avg_train}, Val loss: {loss_avg_val}, Test loss: {loss_avg_test}')

    # save the info

    torch.save({
        'epoch': e,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'losses': errs,
    }, os.path.join(path_mdsav, md_name, f'epc_{e}.pt'))
    
etime = time.time()
print((etime-stime)/60.)


In [ ]:
print((etime-stime)/3600)